# Credit Card Transaction Fraud Detection

In [2]:
# Preprocessing
import numpy as np
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance

# Oversampling and Undersampling
from imblearn.under_sampling import TomekLinks
from imblearn.combine import SMOTETomek

# Model
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# Cross Validation
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

# Figures
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

# For reproducability
RANDOM_STATE = 1

Loading the data.

In [3]:
data = pd.read_csv(r"./fraudTrain.csv")

data.drop(data.columns[0], axis=1, inplace=True)

## Imbalanced Data

In [ ]:
labels = ["Genuine", "Fraud"]

is_fraud_train = data["is_fraud"].value_counts()

plt.pie(is_fraud_train,labels=labels, colors=["lightgreen", "lightcoral"], autopct='%1.1f%%')
plt.title("Genuine vs. Fraud Transactions")
plt.show()

Since 99.4% of the data are genuine transactions, if we do a blind guess of genuine transactions, we will have an accuracy of 99.4%.

## Preprocessing Data

Renaming the columns to be more intuitive and converted the times to be datetimes.

In [ ]:
data.rename(columns={"cc_num": "credit_card",
                     "trans_date_trans_time": "trans_time",
                     "trans_num": "trans_id"},
                inplace=True)

# Turning the transaction time to be numerical and based on time only.
data["trans_time"] = pd.to_datetime(data["trans_time"])
data["dob"] = pd.to_datetime(data["dob"])

data["time"] = pd.to_datetime(data["unix_time"], unit="s")
data["time"] = data["time"].dt.hour



data.head(5)

Next, we convert the date of birth to age and normalize the amount.

In [ ]:
# Normalizing the amount via log(1 + amt)
data["amt"] = np.log1p(data["amt"])

data["age"] = data["trans_time"].dt.year - data["dob"].dt.year


Removing any unecessary features and taking the 4 most recent months as testing data to prevent data leakage.

In [ ]:
# Removed trans_id. Replaced unix_time and trans_date_trans_time with time. Replaced dob with age.
# Also removed private information such as name, dob, address.
features = ['credit_card', 'merchant', 'category', 'amt', 'gender', 'state', 'zip', 'lat', 'long', 'city_pop', 'job', 'age', 'merch_lat', 'merch_long', 'time']
categorical_features = ['credit_card', 'merchant', 'category', 'gender', 'state', 'zip', 'job', 'age']
target = "is_fraud"

# Separates all the data within the most recent 4 months
most_recent_transaction = data["trans_time"].max()
data["month_difference"] = abs(data["trans_time"] - most_recent_transaction) / np.timedelta64(30, "D")

MONTH_DIFF = 4
data_train = data[data.month_difference > MONTH_DIFF]
data_test = data[data.month_difference <= MONTH_DIFF]

Splitting our data into training and testing and encoding the data

In [ ]:
X = data_train[features]
Y = data_train[target]

# Split the data into training and testing. Stratify the data to ensure the split is representative of the actual data.
X_train_all, X_temp_all, Y_train_all, Y_temp_all = train_test_split(X, Y, stratify=Y, train_size=0.8, random_state=RANDOM_STATE)

X_temp_all = pd.concat([X_temp_all, data_test[features]])
Y_temp_all = pd.concat([Y_temp_all, data_test[target]])

# Using ordinal encodings to encode categorical features for data.
ord_enc = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1, dtype=np.float64)

X_train_all.loc[:, categorical_features] = ord_enc.fit_transform(X_train_all[categorical_features])
X_temp_all.loc[:, categorical_features] = ord_enc.transform(X_temp_all[categorical_features])

X_train_all = X_train_all.astype("float64")
X_temp_all = X_temp_all.astype("float64")

X_val_all, X_test_all, Y_val_all, Y_test_all = train_test_split(X_temp_all, Y_temp_all, stratify=Y_temp_all, train_size=0.5, random_state=RANDOM_STATE)

## Feature Importance

To determine which features to select, fit a model to the data.

In [ ]:
model_rf = RandomForestClassifier(random_state=RANDOM_STATE)
model_rf.fit(X_train_all, Y_train_all)

Using the mean decrease in impurity (MDI), plot the feature importance to determine the most important features.

In [ ]:
mdi_importances = model_rf.feature_importances_
mdi_std = np.std([tree.feature_importances_ for tree in model_rf.estimators_], axis=0)

mdi_importances = pd.Series(mdi_importances, index=features)

mdi_fig, mdi_ax = plt.subplots()
mdi_importances.plot.bar(yerr=mdi_std, ax=mdi_ax)
mdi_ax.set_title("Feature importances using MDI")
mdi_ax.set_ylabel("Mean decrease in impurity")
mdi_fig.tight_layout()

Using feature permutation, plot the feature importance to determine the most important features.

In [ ]:
perm_res = permutation_importance(
    model_rf, X_val_all, Y_val_all, n_repeats=10, random_state=RANDOM_STATE, n_jobs=4
)

perm_importances = pd.Series(perm_res.importances_mean, index=features)

perm_fig, perm_ax = plt.subplots()
perm_importances.plot.bar(yerr=perm_res.importances_std, ax=perm_ax)
perm_ax.set_title("Feature importances using permutation on full model")
perm_ax.set_ylabel("Mean accuracy decrease")
perm_fig.tight_layout()
plt.show()

The features `["merchant", "category", "amt", "gender", "city_pop", "job", "age", "merch_lat", "merch_long", "time"]` were found to be the most important.

In [ ]:
selected_features = ["merchant", "category", "amt", "gender", "city_pop", "age", "job", "merch_lat", "merch_long", "time"]

X_train, X_val, X_test = X_train_all[selected_features], X_val_all[selected_features], X_test_all[selected_features]
Y_train, Y_val, Y_test = Y_train_all, Y_val_all, Y_test_all

## Baseline Performance
First, we take a look at a baseline performance using Logistic Regression

In [ ]:
scalar = StandardScaler()
X_train_scaled = scalar.fit_transform(X_train)
X_val_scaled = scalar.transform(X_val)

model_logreg = LogisticRegression(random_state=RANDOM_STATE, max_iter=1000)
model_logreg.fit(X_train_scaled, Y_train)

Y_pred_model_logreg = model_logreg.predict(X_val_scaled)

In [ ]:
print(f"Classification Report: \n{classification_report(Y_val, Y_pred_model_logreg)}\n\n")

print(
    f"Metrics train:\n\tAccuracy score: {accuracy_score(model_logreg.predict(X_train_scaled),Y_train):.4f}\nMetrics test:\n\tAccuracy score: {accuracy_score(model_logreg.predict(X_val_scaled),Y_val):.4f}\n\n")

print("Confusion Matrix:")
cm_display_model = ConfusionMatrixDisplay(confusion_matrix(Y_val, Y_pred_model_logreg), display_labels=["Genuine", "Fraud"])
cm_display_model.plot()

Then, we take a look at the baseline performance of Random Forest

In [ ]:
model_rf.fit(X_train, Y_train)

Y_pred_model_rf = model_rf.predict(X_val)

In [ ]:
print(f"Classification Report: \n{classification_report(Y_val, Y_pred_model_rf)}\n\n")

print(
    f"Metrics train:\n\tAccuracy score: {accuracy_score(model_rf.predict(X_train),Y_train):.4f}\nMetrics test:\n\tAccuracy score: {accuracy_score(model_rf.predict(X_val),Y_val):.4f}\n\n")

print("Confusion Matrix:")
cm_display_model = ConfusionMatrixDisplay(confusion_matrix(Y_val, Y_pred_model_rf), display_labels=["Genuine", "Fraud"])
cm_display_model.plot()

## SMOTE + Tomek Links
Because the data is imbalanced, we combine SMOTE and Tomek Links to resample the data.

In [ ]:
tomek = TomekLinks(sampling_strategy="majority")
smotetomek = SMOTETomek(tomek=tomek, random_state=RANDOM_STATE)

X_train_smotetomek, Y_train_smotetomek = smotetomek.fit_resample(X_train, Y_train)

# Plot the data after resampling
is_fraud_train_smotetomek = Y_train_smotetomek.value_counts()

plt.pie(is_fraud_train_smotetomek,labels=labels, colors=["lightgreen", "lightcoral"], autopct='%1.1f%%')
plt.title("Genuine vs. Fraud Transactions After SMOTE + Tomek")
plt.show()

Running the line of code below will cause the rest of the notebook to use the SMOTE + Tomek data.

In [ ]:
X_train, Y_train = X_train_smotetomek, Y_train_smotetomek

## Model: Decision Tree
Finding and tuning hyperparameters of the Decision Tree using GridSearch.

In [ ]:
max_depths = [3, 5, 7, 10]  # Determines how far down a decision tree goes before stopping
min_splits = [2, 5, 10, 50] # Determines the minimum sample split size before stopping
param_grid_dt = {"max_depth": max_depths,
                            "min_samples_split": min_splits,}

grid_dt = GridSearchCV(DecisionTreeClassifier(random_state=RANDOM_STATE), 
                                  param_grid_dt, 
                                  verbose=3, n_jobs=4)
grid_dt.fit(X_train, Y_train)

print(f"Decision Tree Params: {grid_dt.best_params_}")

```{note}
This may take a while, so not running GridSearch and using the default values given should work.
```

Fitting the data to the Decision Tree with the hyperparameters found.

In [ ]:
try:
    max_depth_dt = grid_dt.best_params_["max_depth"]
    min_samples_split_dt = grid_dt.best_params_["min_samples_split"]
except (NameError, AttributeError):
    max_depth_dt = 10
    min_samples_split_dt = 50

model_dt = DecisionTreeClassifier(
                            min_samples_split=min_samples_split_dt,
                            max_depth=max_depth_dt,
                            random_state=RANDOM_STATE)
model_dt.fit(X_train, Y_train)

Analysis using a classification report, confusion matrix and accuracy.

In [ ]:
Y_pred_dt = model_dt.predict(X_val)

print(f"Classification Report: \n{classification_report(Y_val, Y_pred_dt)}\n")

print(
    f"Metrics train:\n\tAccuracy score: {accuracy_score(model_dt.predict(X_train),Y_train):.4f}\nMetrics test:\n\tAccuracy score: {accuracy_score(model_dt.predict(X_val),Y_val):.4f}\n\n")

print("Confusion Matrix:")
cm_display_dt = ConfusionMatrixDisplay(confusion_matrix(Y_val, Y_pred_dt), display_labels=["Genuine", "Fraud"])
cm_display_dt.plot()

## Model: Random Forest
Becasue Decision Trees are sensitive to changes in data, we use the tree ensemble method Random Forest to mitigate the effects of changes in data.

Finding and tuning hyperparameters of the Random Forest using GridSearch.

In [ ]:
max_depths = [3, 5, 7, 10] # Determines how far down a decision tree goes before stopping
min_splits = [2, 5, 10, 50] # Determines the minimum sample split size before 
n_estimators = [50, 100, 150, 200] # Determines the number of trees used per forest
param_grid_rf = {"max_depth": max_depths,
                            "min_samples_split": min_splits,
                            "n_estimators": n_estimators}

grid_rf = GridSearchCV(RandomForestClassifier(random_state=RANDOM_STATE), 
                                  param_grid_rf, 
                                  verbose=3, n_jobs=4)
grid_rf.fit(X_train, Y_train)

print(f"Random Forest Params: {grid_rf.best_params_}")

```{note}
This will take a while, so not running GridSearch and using the default values given should work.
With the default hyperparameters, it should take 4-12 minutes.
```

Fitting the data to the Random Forest with the hyperparameters found.

In [ ]:
try:
    max_depth_rf = grid_rf.best_params_["max_depth"]
    min_samples_split_rf = grid_rf.best_params_["min_samples_split"]
    n_estimators_rf = grid_rf.best_params_["n_estimators"]
except (NameError, AttributeError):
    max_depth_rf = 10
    min_samples_split_rf = 2
    n_estimators_rf = 150

model_rf = RandomForestClassifier(
                            min_samples_split=min_samples_split_rf,
                            max_depth=max_depth_rf,
                            n_estimators=n_estimators_rf,
                            random_state=RANDOM_STATE)
model_rf.fit(X_train, Y_train)

Analysis using a classification report, confusion matrix and accuracy.

In [ ]:
Y_pred_rf = model_rf.predict(X_val)

print(f"Classification Report: \n{classification_report(Y_val, Y_pred_rf)}\n")

print(
    f"Metrics train:\n\tAccuracy score: {accuracy_score(model_rf.predict(X_train),Y_train):.4f}\nMetrics test:\n\tAccuracy score: {accuracy_score(model_rf.predict(X_val),Y_val):.4f}\n\n")

print("Confusion Matrix:")
cm_display_rf = ConfusionMatrixDisplay(confusion_matrix(Y_val, Y_pred_rf), display_labels=["Genuine", "Fraud"])
cm_display_rf.plot()

### Model: XGBoostClassifier
In order to speedup the algorithm, we use XGBoost.

Finding and tuning hyperparameters of the XGBClassifier using GridSearch.

In [ ]:
max_depths = [3, 5, 7, 10] # Determines how far down a decision tree goes before stopping
gamma = [0.1, 0.3, 0.5, 0.9] # Determines minimum loss reduction for a split
min_child_weight = [1, 3, 5, 7] #  Determines minimum weight needed for a child
param_grid_xgb = {"max_depth": max_depths,
                  "gamma": gamma,
                  "min_child_weight": min_child_weight}

grid_xgb = GridSearchCV(XGBClassifier(random_state=RANDOM_STATE), 
                                  param_grid_xgb, 
                                  verbose=3, n_jobs=4)
grid_xgb.fit(X_train, Y_train)

print(f"Random Forest Params: {grid_xgb.best_params_}")

```{note}
This may take a while, so not running GridSearch and using the default values given should work.
```

Fitting the data to the XGBClassifier with the hyperparameters found.

In [ ]:
try:
    max_depth_xgb = grid_xgb.best_params_["max_depth"]
    gamma_xgb = grid_xgb.best_params_["gamma"]
    min_child_weight_xgb = grid_xgb.best_params_["min_child_weight"]
except (NameError, AttributeError):
    max_depth_xgb = 7
    gamma_xgb = 0.1
    min_child_weight_xgb = 5

model_xgb = XGBClassifier(
                            max_depth=max_depth_xgb,
                            gamma=gamma_xgb,
                            min_child_weight=min_child_weight_xgb,
                            random_state=RANDOM_STATE)

model_xgb.fit(X_train, Y_train)

In [ ]:
Y_pred_xgb = model_xgb.predict(X_val)

print(f"Classification Report: \n{classification_report(Y_val, Y_pred_xgb)}\n")

print(
    f"Metrics train:\n\tAccuracy score: {accuracy_score(model_xgb.predict(X_train),Y_train):.4f}\nMetrics test:\n\tAccuracy score: {accuracy_score(model_xgb.predict(X_val),Y_val):.4f}\n\n")

print("Confusion Matrix:")
cm_display_xgb = ConfusionMatrixDisplay(confusion_matrix(Y_val, Y_pred_xgb), display_labels=["Genuine", "Fraud"])
cm_display_xgb.plot()

## Cross Validation

We perform cross validation by comparing `f1` scores across the three models we trained.

In [ ]:
models = {"Decision Tree": model_dt,
          "Random Forest": model_rf,
          "XGBoost": model_xgb}

strat_kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

best_model = None
best_f1 = None
for name, model in models.items():
    cross_val_res = cross_val_score(model, X_train, Y_train, cv=strat_kf, scoring="f1")

    val_f1 = f1_score(Y_val, model.predict(X_val))

    print(f"{name}: Mean F1 Training = {cross_val_res.mean():.4f}")
    print(f"{name}: F1 Validation = {val_f1:.4f}")

    if best_f1 is None or best_f1 < val_f1:
        best_model = model
        best_f1 = val_f1

```{note}
This may take a while, so not running the cross validation and using the default best model given below should work.
```

We found the best model to be the XGBoosted trees because it has the highest `f1` score.

## Testing

Finally, we perform a final test on the best model.

In [ ]:
try:
    Y_pred_best = best_model.predict(X_test)
except NameError:
    Y_pred_best = model_xgb.predict(X_test)

print(f"Classification Report: \n{classification_report(Y_test, Y_pred_best)}\n")

print(
    f"Metrics test:\n\tAccuracy score: {accuracy_score(best_model.predict(X_test),Y_test):.4f}\n\n")

print("Confusion Matrix:")
cm_display_best = ConfusionMatrixDisplay(confusion_matrix(Y_test, Y_pred_best), display_labels=["Genuine", "Fraud"])
cm_display_best.plot()